In [ ]:
#labels = beh_label
#stimuli = y
#task_mask = nonrest_task_mask
#categories = categories
#mask_filename = mask_vt_file
#masker = masker
#func_filename = func_file 

# ML/DL models:

## Different classifiers

In this notebook I am willing to compare the different classifiers on a visual object recognition decoding task.

In [1]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from nilearn.input_data import NiftiMasker
from nilearn import datasets, plotting, image
from nilearn.image import get_data
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier, RidgeClassifierCV
from sklearn.model_selection import GridSearchCV, LeaveOneGroupOut, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

In [2]:
# Designate input file
haxby_ds = datasets.fetch_haxby(subjects=[4], fetch_stimuli=True)

# 'func' is a list of filenames: one for each subject
func_file = haxby_ds.func[0]

# Standardizing
mask_vt_file = haxby_ds.mask_vt[0]
masker = NiftiMasker(mask_img=mask_vt_file, standardize=True)

# Load the behavioral data that I will predict
beh_label = pd.read_csv(haxby_ds.session_target[0], sep=" ")

#select data
X = masker.fit_transform(func_file)
y = beh_label['labels']

# Identify the resting state
nonrest_task_mask = (y != 'rest')

# Remove the resting state and find names of remaining active labels
categories = y[nonrest_task_mask].unique()

# Extract tags indicating to which acquisition run a tag belongs
session_labels = beh_label['chunks'][nonrest_task_mask]

masked_timecourses = masker.fit_transform(func_file)[nonrest_task_mask]

In [3]:
#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### Decision trees

In [4]:
dtc = tree.DecisionTreeClassifier(criterion='entropy')

#train model
dtc.fit(X_train, y_train)
score = dtc.score(X_test, y_test)

#print classification report
dtc = dtc.predict(X_test)
report = classification_report(y_test, dtc)
print(report)

print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.08      0.17      0.11         6
         cat       0.33      0.25      0.29        12
       chair       0.29      0.40      0.33         5
        face       0.44      0.50      0.47        16
       house       0.33      0.20      0.25        10
        rest       0.67      0.69      0.68        58
    scissors       0.33      0.36      0.35        11
scrambledpix       0.40      0.31      0.35        13
        shoe       0.08      0.07      0.07        15

    accuracy                           0.45       146
   macro avg       0.33      0.33      0.32       146
weighted avg       0.45      0.45      0.44       146

Test score with L1 penalty: 0.4452


### Multinomial Naive Bayes Classifier

In [5]:
#multinomial logistic regression object using L1 penalty
mnb = LogisticRegression(C=50., multi_class='multinomial',
                         penalty='l1', solver='saga', tol=0.1)

#train model
mnb.fit(X_train, y_train)
sparsity = np.mean(mnb.coef_) * 100
score = mnb.score(X_test, y_test)

# print('Best C % .4f' % clf.C_)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

Sparsity with L1 penalty: 0.00%
Test score with L1 penalty: 0.7740


In [6]:
# Prediction accuracy
cv_scores_mnb = cross_val_score(mnb, X_train, y_train, cv=5) 
print(cv_scores_mnb)

# The mean prediction accuracy
classification_accuracy = np.mean(cv_scores_mnb)
classification_accuracy

[0.79770992 0.76628352 0.8045977  0.79310345 0.83524904]


0.7993887280278436

### K-Nearest Neighbours

In [10]:
#shuffle the data and split the sample into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True)

#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

#kneighbors classifier object
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='kd_tree', leaf_size=30, 
                           p=2, metric='minkowski', metric_params=None, n_jobs=None)

#fit model
knn.fit(X_train, y_train)

#response prediction
pred = knn.predict(X_test)

#accuracy
knn.score(X_test, y_test)

#print classification report
knn = knn.predict(X_test)
report = classification_report(y_test, knn)
print(report)

#evaluate accuracy
print("Accuracy on test set: %0.3f%%"%(accuracy_score(y_test, pred)*100))

              precision    recall  f1-score   support

      bottle       0.57      1.00      0.73         8
         cat       0.60      0.75      0.67         8
       chair       0.62      0.62      0.62        13
        face       1.00      0.80      0.89        15
       house       0.83      0.83      0.83        12
        rest       0.89      0.77      0.83        62
    scissors       0.25      0.60      0.35         5
scrambledpix       0.82      0.75      0.78        12
        shoe       0.88      0.64      0.74        11

    accuracy                           0.76       146
   macro avg       0.72      0.75      0.71       146
weighted avg       0.81      0.76      0.77       146

Accuracy on test set: 76.027%


In [11]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='kd_tree', leaf_size=30, 
                           p=2, metric='minkowski', metric_params=None, n_jobs=None)

knn.fit(X_train, y_train)

# Prediction accuracy
cv_scores_knn = cross_val_score(knn, X_train, y_train, cv=5) 
print(cv_scores_knn)

# The mean prediction accuracy
classification_accuracy_knn = np.mean(cv_scores_knn)
classification_accuracy_knn

[0.76335878 0.72030651 0.68582375 0.70114943 0.7164751 ]


0.717422713579597

### Neural Networks

In [12]:
#multinomial logistic regression object using L1 penalty
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5, 2),
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=1,
              solver='lbfgs', tol=0.0001, validation_fraction=0.1, 
              verbose=False, warm_start=False)

#train model
nn.fit(X_train, y_train)
score = nn.score(X_test, y_test)

#print classification report
nn = nn.predict(X_test)
report = classification_report(y_test, nn)
print(report)

print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.33      0.12      0.18         8
         cat       0.17      0.50      0.25         8
       chair       0.69      0.69      0.69        13
        face       0.00      0.00      0.00        15
       house       0.79      0.92      0.85        12
        rest       0.93      0.84      0.88        62
    scissors       0.00      0.00      0.00         5
scrambledpix       0.55      0.50      0.52        12
        shoe       0.38      0.55      0.44        11

    accuracy                           0.61       146
   macro avg       0.43      0.46      0.42       146
weighted avg       0.62      0.61      0.61       146

Test score with L1 penalty: 0.6096


/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
nn = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

MLPClassifier(activation='logistic', alpha=1e-05, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False,
              epsilon=1e-08, hidden_layer_sizes=(5, 2),
              max_iter=200, momentum=0.9, n_iter_no_change=10,
              nesterovs_momentum=True, power_t=0.5, random_state=1,
              solver='lbfgs', tol=0.0001, validation_fraction=0.1, 
              verbose=False, warm_start=False)

#train model
nn.fit(X_train, y_train)

# Prediction accuracy
cv_scores_nn = cross_val_score(nn, X_train, y_train, cv=5) 
print(cv_scores_nn)

# The mean prediction accuracy
classification_accuracy_nn = np.mean(cv_scores_nn)
classification_accuracy_nn

/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed t

[0.58015267 0.60536398 0.54022989 0.60536398 0.47509579]


/home/srastegarnia/miniconda3/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


0.5612412623204937

### Multinomial Logistic Regression

In [18]:
logistic_50 = LogisticRegression(C=50., multi_class='multinomial',
                     penalty='elasticnet', solver='saga', tol=0.1, l1_ratio=0.4)

#train model
logistic_50.fit(X_train, y_train)
sparsity = np.mean(logistic_50.coef_ == 0) * 100
score = logistic_50.score(X_test, y_test)

#print classification report
logistic_50 = logistic_50.predict(X_test)
report = classification_report(y_test, logistic_50)
print(report)

print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.62      0.62      0.62         8
         cat       0.60      0.38      0.46         8
       chair       0.73      0.85      0.79        13
        face       0.79      1.00      0.88        15
       house       0.85      0.92      0.88        12
        rest       0.96      0.84      0.90        62
    scissors       0.40      0.80      0.53         5
scrambledpix       0.92      0.92      0.92        12
        shoe       0.80      0.73      0.76        11

    accuracy                           0.82       146
   macro avg       0.74      0.78      0.75       146
weighted avg       0.84      0.82      0.82       146

Sparsity with L1 penalty: 0.00%
Test score with L1 penalty: 0.8219


In [19]:
# Prediction accuracy
cv_scores_logistic_50 = cross_val_score(knn, X_train, y_train, cv=5) 
print(cv_scores_logistic_50)

# The mean prediction accuracy
classification_accuracy_logistic_50 = np.mean(cv_scores_logistic_50)
classification_accuracy_logistic_50

[0.76335878 0.72030651 0.68582375 0.70114943 0.7164751 ]


0.717422713579597

### Support Vector Machines

In [22]:
#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

svm.fit(X_train, y_train)

#accuracy
score = svm.score(X_test, y_test)

#print classification report
svm = svm.predict(X_test)
report = classification_report(y_test, svm)
print(report)

print("Test score with L1 penalty: %.4f" % score)

              precision    recall  f1-score   support

      bottle       0.45      0.62      0.53         8
         cat       0.67      0.50      0.57         8
       chair       0.79      0.85      0.81        13
        face       0.93      0.87      0.90        15
       house       0.85      0.92      0.88        12
        rest       0.90      0.84      0.87        62
    scissors       0.50      0.80      0.62         5
scrambledpix       0.77      0.83      0.80        12
        shoe       0.89      0.73      0.80        11

    accuracy                           0.81       146
   macro avg       0.75      0.77      0.75       146
weighted avg       0.82      0.81      0.81       146

Test score with L1 penalty: 0.8082


In [23]:
#standarize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

svm = SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

svm.fit(X_train, y_train)

# Prediction accuracy
cv_scores_logistic_50 = cross_val_score(knn, X_train, y_train, cv=5) 
print(cv_scores_logistic_50)

# The mean prediction accuracy
classification_accuracy_logistic_50 = np.mean(cv_scores_logistic_50)
classification_accuracy_logistic_50

[0.76335878 0.72030651 0.68582375 0.70114943 0.7164751 ]


0.717422713579597

In [24]:
# A dictionary, to hold all our classifiers
classifiers = {'SVC': svm,
               'LogisticRegression': logistic_50,
               'KNeighborsClassifier':knn,
               'DecisionTreeClassifier': dtc,
               'MLPClassifier': nn,
               'Multinomial Naive Bayes': mnb,
               }

In [ ]:
# Make a data splitting object for cross validation
cv = LeaveOneGroupOut()

classifiers_scores = {}

for classifier_name, classifier in sorted(classifiers.items()):
    classifiers_scores[classifier_name] = {}
    print(70 * '_')

    for category in categories:
        classification_target = y[nonrest_task_mask].isin([category])
        t0 = time.time()
        classifiers_scores[classifier_name][category] = cross_val_score(
            classifier,
            masked_timecourses,
            classification_target,
            cv=5, verbose=1
        )

        print(
            "%10s: %14s -- scores: %1.2f +- %1.2f, time %.2fs" %
            (
                classifier_name,
                category,
                classifiers_scores[classifier_name][category].mean(),
                classifiers_scores[classifier_name][category].std(),
                time.time() - t0,
            ),
        )

In [ ]:
# Make a data splitting object for cross validation
cv = LeaveOneGroupOut()

classifiers_scores = {}

for classifier_name, classifier in sorted(classifiers.items()):
    classifiers_scores[classifier_name] = {}
    print(70 * '_')

    for category in categories:
        classification_target = y[nonrest_task_mask].isin([category])
        t0 = time.time()
        classifiers_scores[classifier_name][category] = cross_val_score(
            classifier,
            X_train,
            y_train,
            cv=5, verbose=1
        )

        print(
            "%10s: %14s -- scores: %1.2f +- %1.2f, time %.2fs" %
            (
                classifier_name,
                category,
                classifiers_scores[classifier_name][category].mean(),
                classifiers_scores[classifier_name][category].std(),
                time.time() - t0,
            ),
        )

In [ ]:
plt.figure()

tick_position = np.arange(len(categories))
plt.xticks(tick_position, categories, rotation=45)

for color, classifier_name in zip(
        ['b', 'c', 'm', 'g', 'y', 'k', '.5', 'r', '#ffaaaa'],
        sorted(classifiers)):
    score_means = [classifiers_scores[classifier_name][category].mean()
                   for category in categories]
    plt.bar(tick_position, score_means, label=classifier_name,
            width=.11, color=color)
    tick_position = tick_position + .09

plt.ylabel('Classification accurancy (f1 score)')
plt.xlabel('Visual stimuli category')
plt.ylim(ymin=0)
plt.legend(loc='lower center', ncol=3)
plt.title(
    'Category-specific classification accuracy for different classifiers')
plt.tight_layout(